In [40]:
import numpy as np
import dpkt
from tqdm import tqdm

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
# from open3d import JVisualizer 
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
# op3.visualization.webrtc_server.enable_webrtc()
from open3d.web_visualizer import draw
from VisulizerTools import *
from tqdm import tqdm_notebook
import time
from sklearn.cluster import DBSCAN
# +80

In [160]:
file_path = r'D:\LiDAR_Data\MidTown\PCAP_BCK\Liberty\Days\2021-12-12-16-0-0.pcap'
fpcap = open(file_path, 'rb')
lidar_reader = dpkt.pcap.Reader(fpcap)
loader = frame_gen(lidar_reader)

In [151]:
Reads = []
while True:
    try:
        read = next(lidar_reader)
        Reads.append(read)
    except dpkt.NeedData as e1:
        print('ND')
        continue
    except StopIteration as e2:
        print('SI')
        break 

ND
SI


In [158]:
len(Reads[0][1])

1248

In [161]:
Td_maps = []
for td in tqdm(loader):
    if td is not None:
        Td_maps.append(td)
    else:
        break

1119it [00:44, 25.32it/s]


In [128]:
threshold_map = np.load(r'D:\LiDAR_Data\test\Calibration\bck_map.npy')

In [163]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[-1]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_1')

In [189]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[20]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_3')

In [180]:
Foreground_map = np.full(Td_maps[0].shape,True)
Labeling_map = np.ones_like(Foreground_map)
pcd = get_pcd_colored(Td_maps[0],Labeling_map)

In [181]:
draw(pcd)

WebVisualizer(window_uid='window_0')

In [123]:
Reads = []
while True:
    try:
        Reads.append(next(lidar_reader))
    except StopIteration:
        print('Done')
        break

Done


In [42]:
laser_id = np.full((32,12),np.arange(32).reshape(-1,1).astype('int'))
Data_order = np.array([[-25,1.4],[-1,-4.2],[-1.667,1.4],[-15.639,-1.4],
                                    [-11.31,1.4],[0,-1.4],[-0.667,4.2],[-8.843,-1.4],
                                    [-7.254,1.4],[0.333,-4.2],[-0.333,1.4],[-6.148,-1.4],
                                    [-5.333,4.2],[1.333,-1.4],[0.667,4.2],[-4,-1.4],
                                    [-4.667,1.4],[1.667,-4.2],[1,1.4],[-3.667,-4.2],
                                    [-3.333,4.2],[3.333,-1.4],[2.333,1.4],[-2.667,-1.4],
                                    [-3,1.4],[7,-1.4],[4.667,1.4],[-2.333,-4.2],
                                    [-2,4.2],[15,-1.4],[10.333,1.4],[-1.333,-1.4]
                                    ])
omega = Data_order[:,0]

In [43]:
def read_uint32(data, idx):
    return data[idx] + data[idx+1]*256 + data[idx+2]*256*256 + data[idx+3]*256*256*256
def read_firing_data(data):
    block_id = data[0] + data[1]*256
    azimuth = (data[2] + data[3] * 256) / 100 # degree
    firings = data[4:].reshape(32, 3) 
    distances = firings[:, 0] + firings[:, 1] * 256 # mm 
    intensities = firings[:, 2] # 0-255
    return distances, intensities, azimuth #(1,0)

def calc_timing_offsets():
    timing_offsets = np.zeros((32,12))  # Init matrix
    # constants
    full_firing_cycle = 55.296  # μs
    single_firing = 2.304  # μs
    # compute timing offsets
    for x in range(12):
        for y in range(32):
            dataBlockIndex = (x * 2) + int((y / 16))
            dataPointIndex = y % 16
            timing_offsets[y][x] = (full_firing_cycle * dataBlockIndex) +(single_firing * dataPointIndex)
    return np.array(timing_offsets).T

def calc_precise_azimuth(azimuth_per_block):

    # block_number: how many blocks are required to be processed 

    org_azi = azimuth_per_block.copy()
    precision_azimuth = []
    # iterate through each block
    for n in range(len(org_azi)): # n=0..11
        azimuth = org_azi.copy()
        try:
            # First, adjust for an Azimuth rollover from 359.99° to 0°
            if azimuth[n + 1] < azimuth[n]:
                azimuth[n + 1] += 360

            # Determine the azimuth Gap between data blocks
            azimuth_gap = azimuth[n + 1] - azimuth[n]
        except:
            if azimuth[n] < azimuth[n-1]:
                azimuth[n] += 360
            azimuth_gap = azimuth[n] - azimuth[n-1]

        factor = azimuth_gap / 32.
        k = np.arange(32)
        precise_azimuth = azimuth[n] + factor * k
        precision_azimuth.append(precise_azimuth)

    precision_azimuth = np.array(precision_azimuth).T
    return precision_azimuth # 32 * 12

def parse_one_packet(data):
    data = np.frombuffer(data, dtype=np.uint8).astype(np.uint32)
    blocks = data[0:1200].reshape(12, 100)
    # Timestamp = self.read_uint32(data[1200:1204],0)
    distances = []#12*32
    intensities = []#12*32
    azimuth_per_block = [] #(12,0)
    # iteratie through each block
    for i, blk in enumerate(blocks):
        dists, intens, angles = read_firing_data(blk)
        distances.append(dists) #12*32
        intensities.append(intens) #12*32
        azimuth_per_block.append(angles)

    azimuth_per_block = np.array(azimuth_per_block).T
    distances = 4/1000*np.array(distances).T # 32,12
    intensities = np.array(intensities).T # 32,12

    return distances,intensities, azimuth_per_block # 12*0

def cal_angle_diff(advance_angle,lagging_angle):
    if advance_angle < lagging_angle:
        return advance_angle + 360 - lagging_angle
    else:
        return advance_angle - lagging_angle

In [159]:
def frame_gen(lidar_reader):
    frame_initial_azimuth = -1
    cur_azimuth = -1
    culmulative_azimuth = 0
    culmulative_azimuth_values = []
    culmulative_laser_ids = []
    culmulative_distances = []
#     Iteration_stop_sign = False
    while True:
        Td_map = np.zeros((32,1800))
        Intens_map = np.zeros((32,1800))
        while True:
            try:
                ts,buf = next(lidar_reader)
            except (StopIteration,dpkt.NeedData) as e1:
                yield None
            if len(buf) == 1248:
                eth = dpkt.ethernet.Ethernet(buf)
                data = eth.data.data.data
                packet_status = eth.data.data.sport
                if packet_status == 2368:
                    if len(data)<1206:
                        culmulative_azimuth += 2.4
                        continue
                    # 32,12, 32,12, （12，） azi
                    distances,intensities,azimuth_per_block = parse_one_packet(data)
                    azimuth = calc_precise_azimuth(azimuth_per_block) # 32 x 12

    #                 print(azimuth_per_block)
                    if frame_initial_azimuth == -1: #initialization
                        frame_initial_azimuth = azimuth_per_block[0]
                        cur_azimuth = azimuth_per_block[-1]
                        culmulative_azimuth_values.append(azimuth)
                        culmulative_laser_ids.append(laser_id)
                        culmulative_distances.append(distances)
                        # azimuth_ind = azimuth_ind.flatten()
                        # Td_map[laser_id,azimuth_ind] = distances.flatten()
                        culmulative_azimuth += cal_angle_diff(azimuth_per_block[-1],azimuth_per_block[0])
                    else:
                        # non-initialization
                        diff = cal_angle_diff(azimuth_per_block[-1],cur_azimuth)
                        culmulative_azimuth += diff 

                        if culmulative_azimuth > 360: 

                            culmulative_azimuth_values.append(azimuth)
                            culmulative_laser_ids.append(laser_id)
                            culmulative_distances.append(distances)

                            culmulative_azimuth_values = np.concatenate(culmulative_azimuth_values,axis = 1)
                            culmulative_azimuth_values += Data_order[:,1].reshape(-1,1)
                            culmulative_laser_ids = np.concatenate(culmulative_laser_ids,axis = 1).flatten()
                            culmulative_distances = np.concatenate(culmulative_distances,axis = 1).flatten()

                            culmulative_azimuth_inds = np.around(culmulative_azimuth_values/0.2).astype('int').flatten()
                            culmulative_azimuth_inds[culmulative_azimuth_inds > 1799] -= 1800
                            culmulative_azimuth_inds[culmulative_azimuth_inds < 0 ] += 1800

                            Td_map[culmulative_laser_ids,culmulative_azimuth_inds] = culmulative_distances
                            culmulative_azimuth = 0
                            culmulative_azimuth_values = []
                            culmulative_laser_ids = []
                            culmulative_distances = []

                            cur_azimuth = azimuth_per_block[-1]

                            yield Td_map[np.argsort(omega),:] #32*1800

                        else:
                            culmulative_azimuth_values.append(azimuth)
                            culmulative_laser_ids.append(laser_id)
                            culmulative_distances.append(distances)
                            cur_azimuth = azimuth_per_block[-1]

In [1]:
from math import sqrt
from joblib import Parallel, delayed  
from tqdm import tqdm  

In [2]:
result = Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in tqdm(range(100000)))


100%|██████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 169067.01it/s]
